[Optimizing Model Parameters](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [3]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# Optimization Loop

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
""" Training loop
1. Call optimizer.zero_grad() to reset the gradients of model parameters.
Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

2. Backpropagate the prediction loss with a call to loss.backward().
PyTorch deposits the gradients of the loss w.r.t. each parameter.

3. Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.
"""
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()        # 2
        optimizer.step()       # 3
        optimizer.zero_grad()  # 1.

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.289877  [   64/60000]
loss: 2.279617  [ 6464/60000]
loss: 2.259396  [12864/60000]
loss: 2.254736  [19264/60000]
loss: 2.232304  [25664/60000]
loss: 2.197944  [32064/60000]
loss: 2.203627  [38464/60000]
loss: 2.160480  [44864/60000]
loss: 2.157311  [51264/60000]
loss: 2.141268  [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 2.118274 

Epoch 2
-------------------------------
loss: 2.121143  [   64/60000]
loss: 2.114586  [ 6464/60000]
loss: 2.042174  [12864/60000]
loss: 2.067909  [19264/60000]
loss: 2.013190  [25664/60000]
loss: 1.937466  [32064/60000]
loss: 1.975026  [38464/60000]
loss: 1.875986  [44864/60000]
loss: 1.892708  [51264/60000]
loss: 1.837000  [57664/60000]
Test Error: 
 Accuracy: 57.3%, Avg loss: 1.810992 

Epoch 3
-------------------------------
loss: 1.834214  [   64/60000]
loss: 1.811267  [ 6464/60000]
loss: 1.671709  [12864/60000]
loss: 1.740592  [19264/60000]
loss: 1.630152  [25664/60000]
loss: 1.573727  [32064/600